In [1]:
import boto3
import sagemaker
import os
import numpy as np

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# location of stored sata
data_dir = 'modelling'
# S3 location
bucket = sagemaker_session.default_bucket()
# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/audio_emotion_detection'

## Load and deploy saved model

An AWS blogpost about deployment of a keras model on SageMaker (https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/) gave me a good general idea how to proceed. First, I changed the training approach to save model in a format enabling deployment on SageMaker. The blogpost described a workaround to deploy a model, by using sagemaker.tensorflow.model.TensorFlowModel and simulating its creation with an empty train.py file. As I discovered from the Sagemaker documentation, there is now a better way to deploy directly from model artifacts (https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploying-directly-from-model-artifacts).

In [3]:
!ls export/1

saved_model.pb	variables


In [4]:
#Tar the entire directory and upload to Amazon S3
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix=prefix)

## Deploy the trained model

TensorFlow Serving Model
https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html

Specification of *framework_version='2.0.0'* alighns the TensorFlow version of the trained model with the version of the serving model. Otherwise, an error was thrown.


In [5]:
from sagemaker.tensorflow.serving import Model
model = Model(model_data = 's3://' + bucket + '/' + prefix + '/' + 'model.tar.gz', 
              role = role,
             framework_version='2.0.0')

In [6]:
%%time
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m4.xlarge')

--------------------------------------------------------------------------------------!CPU times: user 462 ms, sys: 58.3 ms, total: 520 ms
Wall time: 7min 13s


In [7]:
predictor.endpoint

'tensorflow-inference-2019-12-30-19-32-10-123'

## Making predictions

Sending a preprocessed sound data (MFCC feature) to the endpoint results in probability prediction for each of the classes. The endpoint accepts different formats and an appropriate should be chosen to be used with the lambda function.

JSON-formats:

In [8]:
mfcc = [-626.9103, 82.518814, 13.619398, 17.021456, -4.8949747, -4.666426, -17.72673, -9.443328, -11.367254, -10.0687, -5.5265326, -5.759393, -9.602776, -6.8477325, -6.8149705, -6.11381, -4.946893, -3.0180538, -4.766686, -6.5694504, -6.881639, -5.4145484, -8.107064, -3.1986628, -5.3452864, -3.0117583, -4.903243, -2.4027758, -2.3773522, 1.0952495]

In [9]:
input = {
  'instances': [mfcc]
}

In [10]:
result = predictor.predict(input)
result

{'predictions': [[6.27396901e-19,
   8.07409707e-24,
   1.41264081e-05,
   0.994807065,
   0.00517883385]]}

In [11]:
input = [mfcc]

In [12]:
result = predictor.predict(input)
result

{'predictions': [[6.27396901e-19,
   8.07409707e-24,
   1.41264081e-05,
   0.994807065,
   0.00517883385]]}

CSV (comma-separated values) fortmat:

In [13]:
from sagemaker.tensorflow.serving import Predictor
# create a Predictor with JSON serialization
predictor = Predictor(predictor.endpoint, serializer=sagemaker.predictor.csv_serializer)

# CSV-formatted string input
input = mfcc

result = predictor.predict(input)
result

{'predictions': [[6.27396901e-19,
   8.07409707e-24,
   1.41264081e-05,
   0.994807065,
   0.00517883385]]}

In [14]:
result = result['predictions'][0]

Transform into a label prediction:

In [15]:
class_mapping_dict = {'angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4}

In [16]:
# get the index of a maximum value
max_idx = np.argmax(result)
# get key of a value equal to the index of a maximum value
# sourced from https://stackoverflow.com/questions/8023306/get-key-by-value-in-dictionary
result_label = list(class_mapping_dict.keys())[list(class_mapping_dict.values()).index(max_idx)]
result_label

'neutral'

## Invoke endpoint as by a Lambda function

In [17]:
input = {'instances': [mfcc]}

In [18]:
import json
json.dumps(input)

'{"instances": [[-626.9103, 82.518814, 13.619398, 17.021456, -4.8949747, -4.666426, -17.72673, -9.443328, -11.367254, -10.0687, -5.5265326, -5.759393, -9.602776, -6.8477325, -6.8149705, -6.11381, -4.946893, -3.0180538, -4.766686, -6.5694504, -6.881639, -5.4145484, -8.107064, -3.1986628, -5.3452864, -3.0117583, -4.903243, -2.4027758, -2.3773522, 1.0952495]]}'

In [19]:
import boto3

runtime = boto3.Session().client('sagemaker-runtime')

In [20]:
# Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
response = runtime.invoke_endpoint(EndpointName = predictor.endpoint, # The name of the endpoint
                                       ContentType = 'application/json', # The data format that is expected
                                       Body = json.dumps(input) # The preprocessed audio file
                                  )

In [21]:
response

{'ResponseMetadata': {'RequestId': 'b8ec8a9c-b411-4475-b308-df001f69913c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b8ec8a9c-b411-4475-b308-df001f69913c',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Mon, 30 Dec 2019 19:40:23 GMT',
   'content-type': 'application/json',
   'content-length': '106'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7ffa83afa208>}

In [22]:
# The response is an HTTP response whose body contains the result of our inference
result = response['Body'].read().decode('utf-8')
result

'{\n    "predictions": [[6.27396901e-19, 8.07409707e-24, 1.41264081e-05, 0.994807065, 0.00517883385]\n    ]\n}'

In [23]:
# thansform to json
result = json.loads(result)

In [24]:
# extract an array of probability predictions
result = result['predictions'][0]
    
# get the index of a maximum value
max_idx = np.argmax(result)
# get key of a value equal to the index of a maximum value
# sourced from https://stackoverflow.com/questions/8023306/get-key-by-value-in-dictionary
result_label = list(class_mapping_dict.keys())[list(class_mapping_dict.values()).index(max_idx)]

In [25]:
result_label

'neutral'

In [26]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))


In [27]:
delete_endpoint(predictor)

Deleted tensorflow-inference-2019-12-30-19-32-10-123
